In [2]:
!pip install torchnet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Created wheel for torchnet: filename=torchnet-0.0.4-py3-none-any.whl size=29742 sha256=818c9ad4974faa08490ad08ecb35a97c02a5eb05aad1ed6ecf1129e82ce13f19
  Stored in directory: /Users/dirceusilva/Library/Caches/pip/wheels/65/d2/1e/0c2519d1837089fe374a60ce10e120c555b9d77110a49f027f
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408218 sha256=100bad8217fa71897b293023bcffef6bdf46b3c881ad9b7aa8ed8dc657208ca1
  Stored in directory: /Users/dirceusilva/Library/Caches/pip/wheels/58/9e/14/30f7cc4dafdd4d602fb00ca33c6edd1424fc0f5df10a02e060
Successfully built torchnet visdom


In [3]:
import os
import torch
from cqt_loader import *
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import models
from config import opt
from torchnet import meter
from tqdm import tqdm
import numpy as np
import time
import torch.nn.functional as F
import torch
import torch.nn as nn
from utility import *

In [5]:

import models.CQTNet


modelname = 'CQTNet'
load_latest = True
load_model_path = 'check_points/CQTNet.pth'
#opt._parse(kwargs)

model = models.CQTNet.CQTNet()
print(model)

CQTNet(
  (features): Sequential(
    (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
    (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (pool3)

In [6]:
features = nn.Sequential(*list(model.children())[:-1])
print(features)

Sequential(
  (0): Sequential(
    (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
    (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (pool3): M

In [ ]:
in_features = vgg16._modules['classifier'][-1].in_features
out_features = 80
vgg16._modules['classifier'][-1] = nn.Linear(in_features, out_features, bias=True)
print(vgg16._modules['classifier'])

In [ ]:
#if opt.load_latest is True:
#    model.load_latest(opt.notes)
elif opt.load_model_path:
    model.load(opt.load_model_path)
model.to(opt.device)